In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gmap_api_key


In [2]:
csv_file = os.path.join(os.getcwd(), 'output_data', 'cities.csv')
cities_df = pd.read_csv(csv_file)

In [3]:
cities_df

,city,lat,lng,temp,humidity,cloudiness,windspeed,date
0,vaini,-21.2000,-175.2000,80.76,78,20,13.80,1624329259
1,cabo san lucas,22.8909,-109.9124,78.42,68,1,26.46,1624329005
2,lavrentiya,65.5833,-171.0000,48.24,71,97,4.21,1624329260
3,qaqortoq,60.7167,-46.0333,46.90,89,100,2.10,1624329260
4,lucapa,-8.4192,20.7447,61.16,52,53,4.72,1624329260
...,...,...,...,...,...,...,...,...
539,chingirlau,51.0950,54.0814,75.58,48,0,0.63,1624329401
540,makokou,0.5738,12.8642,69.03,98,100,2.13,1624329401
541,torrevieja,37.9787,-0.6822,73.09,45,0,5.01,1624329401
542,svetlyy,54.6750,20.1347,68.05,85,1,6.13,1624329402


In [4]:
gmaps.configure(api_key=gmap_api_key)

In [5]:
# Customize the size of the figure
figure_layout = {
    "width": "400px",
    "height": "300px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}
fig = gmaps.figure(layout=figure_layout)
humidity = cities_df['humidity'].astype(float)

In [6]:
# Assign the marker layer to a variable

marker_locations =cities_df[['lat', 'lng']]
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# Add the layer to the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [8]:
perfect_cities = cities_df.loc[(cities_df["temp"]>70)&(cities_df['temp']<80)&(cities_df['windspeed']<10)&(cities_df['cloudiness']==0)].dropna()

In [10]:
hotel_df = perfect_cities[["city","lat","lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-10-a961ac32f278>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,city,lat,lng,Hotel Name
7,riyadh,24.6877,46.7219,
108,avarua,-21.2078,-159.7750,
159,faanui,-16.4833,-151.7500,
280,maceio,-9.6658,-35.7353,
341,marsaxlokk,35.8419,14.5431,
405,tonekabon,36.8163,50.8738,
431,vaitape,-16.5167,-151.7500,
469,surab,28.4914,66.2585,
539,chingirlau,51.0950,54.0814,
541,torrevieja,37.9787,-0.6822,


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [16]:
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': gmap_api_key
    }
for index, row in hotel_df.iterrows():
    lat=row['lat']
    lng=row['lng']
    params['location']=f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    address=requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"]=address["results"][0]["name"]
    except:
        print("No hotels nearby")
        pass

E:\downloads\anaconda\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
No hotels nearby


In [17]:
hotel_df

,city,lat,lng,Hotel Name
7,riyadh,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
108,avarua,-21.2078,-159.7750,Paradise Inn
159,faanui,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
280,maceio,-9.6658,-35.7353,Poutur Pousada
341,marsaxlokk,35.8419,14.5431,Water's Edge
405,tonekabon,36.8163,50.8738,Hotel Sadra
431,vaitape,-16.5167,-151.7500,Conrad Bora Bora Nui
469,surab,28.4914,66.2585,Bismillah Hotel
539,chingirlau,51.0950,54.0814,
541,torrevieja,37.9787,-0.6822,Hotel Fontana Plaza


In [19]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]


In [20]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…